In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

#### 변경 셀 사이즈 및 KATECH 좌료 원점 ####
cell_size = 500
x_base,y_base = 400000,600000

def cv_cell(x,y):  ### 셀 사이즈 변경 함수 정의 ###
    a = ((x - x_base)//cell_size)*cell_size + (cell_size//2 -25) + x_base
    b = ((y - y_base)//cell_size)*cell_size + (cell_size//2 -25) + y_base
    c = (a//100)*10000 + (b//100)
    return(c,a,b)

In [2]:
path = f"X:\D000. 유동인구 데이터\D800. AI바우처_서울시" #원천데이터 파일위치
os.chdir(path)
file_list = os.listdir(path)
f_list = [file for file in file_list if file.endswith(".csv")]
print(f_list)

['SEOUL_202010.csv', 'SEOUL_202011.csv']


In [3]:
col_name = ['ID', 'X','Y','TIMEZN_CD','M00','M10','M15',
            'M20','M25','M30','M35','M40','M45','M50','M55','M60','M65','M70',
            'F00','F10','F15','F20','F25','F30','F35','F40','F45','F50','F55',
            'F60','F65','F70','TOTAL','ADMI_CD','ETL_YMD']

In [4]:
for filename in tqdm(f_list):
    print(filename)

##########################################################

    chunksize = 75000000  ### 1번에 처리할 행의 갯수 정의 ###
    
##########################################################    
    
    cnt_list=[]
    for cnt, celldf in tqdm(enumerate(pd.read_csv(filename, header = 0, names=col_name, encoding = "UTF-8", chunksize=chunksize))):
        print(cnt, celldf.shape)
         
        a=[]
        cellar = celldf[['X','Y']].values
        for rows in cellar:
            a.append(cv_cell(rows[0], rows[1]))
        
        celldf[['ID','X','Y']] = a
        celldf = celldf.groupby(['ID', 'X', 'Y', 'TIMEZN_CD','ETL_YMD'], as_index = False).sum()
        columns = ['ID', 'X', 'Y', 'TIMEZN_CD', 'M00', 'M10', 'M15', 'M20', 'M25', 'M30','M35', 'M40', 'M45', 'M50', 'M55', 'M60', 'M65', 'M70', 
                   'F00', 'F10','F15', 'F20', 'F25', 'F30', 'F35', 'F40', 'F45', 'F50', 'F55', 'F60','F65', 'F70', 'TOTAL', 'ETL_YMD']
        celldf = celldf[columns]
        
        cnt_filename = filename[:-4]+str(cnt)+filename[-4:]
        cnt_list.append(cnt_filename)
        celldf.to_csv(cnt_filename, encoding = "UTF-8" ,index = False)
    
    celldf = pd.DataFrame(columns=columns)
    print('### 파일을 통합합니다. ###')
    
    for filename in cnt_list:
        adf = pd.read_csv(filename, header = 0, encoding = "UTF-8")
        celldf = pd.concat([celldf,adf])
        os.remove(filename)
        
    celldf = celldf.groupby(['ID', 'X', 'Y', 'TIMEZN_CD','ETL_YMD'], as_index = False).sum()
    celldf = celldf[columns]
    
    mg_filename = './500/CV500_'+filename[:-5]+'.csv'
    print(mg_filename)
    celldf.to_csv(mg_filename, encoding = "UTF-8" ,index = False)

  0%|          | 0/2 [00:00<?, ?it/s]

SEOUL_202010.csv


0it [00:00, ?it/s]

0 (75000000, 35)
1 (49191785, 35)
### 파일을 통합합니다. ###
./500/CV500_SEOUL_202010.csv
SEOUL_202011.csv


0it [00:00, ?it/s]

0 (75000000, 35)
1 (45001697, 35)
### 파일을 통합합니다. ###
./500/CV500_SEOUL_202011.csv
